In [1]:
import numpy as np
import pandas as pd

In [2]:
main_folder = "data/proyectos/"
ref_folder = "data/ref/"
extra_folder =  "data/extra/"
years = [year for year in range(2008,2020,1)]
extras = ["beneficiario","disciplina","participante"]
refs = ["disciplina","estado_proyecto","moneda"]

In [3]:
def get_main_data(main_folder):    
    data = pd.DataFrame()
    for year in years:    
        current_data = pd.read_csv(f"{main_folder}proyectos_{year}.csv",delimiter = ";")
        current_data["year"] = year
        current_data["duration"] = pd.to_datetime(current_data.fecha_finalizacion) - pd.to_datetime(current_data.fecha_inicio)         
        current_data["amount_requested_awarded_difference"] = current_data.monto_total_adjudicado  - current_data.monto_total_solicitado
        current_data.drop(columns = ["fecha_inicio","fecha_finalizacion","monto_financiado_solicitado","monto_financiado_adjudicado","codigo_identificacion","fondo_anpcyt","tipo_proyecto_id"], inplace = True)
        data = pd.concat([data, current_data], ignore_index=True)
    return data

In [4]:
def get_extra(data,extra_folder,extras):
    #beneficiario
    current_data = pd.read_csv(f"{extra_folder}proyecto_{extras[0]}.csv",delimiter = ";")
    current_data = current_data[["proyecto_id","organizacion_id"]].groupby("proyecto_id").count().rename(columns = {"organizacion_id":"amount_of_benefactors"})
    data = pd.merge(data, current_data, on='proyecto_id', how='inner')

    #disiplina
    current_data = pd.read_csv(f"{extra_folder}proyecto_{extras[1]}.csv",delimiter = ";")
    data = pd.merge(data, current_data, on='proyecto_id', how='inner') 

    #participantes
    current_data = pd.read_csv(f"{extra_folder}proyecto_{extras[2]}.csv",delimiter = ";")
    grouped = current_data.groupby('proyecto_id')['funcion_id'].agg(list).reset_index()
    unique_funcion_ids = sorted(set(current_data['funcion_id']))
    counts = {f'funcion_id_{funcion_id}': [] for funcion_id in unique_funcion_ids}
    for _, row in grouped.iterrows():
        for funcion_id in unique_funcion_ids:
            counts[f'funcion_id_{funcion_id}'].append(row['funcion_id'].count(funcion_id))
    df_counts = pd.DataFrame(counts)
    df_counts['proyecto_id'] = grouped['proyecto_id']
    function_mapping = {"funcion_id_1":"directores","funcion_id_2":"investigadores","funcion_id_3":"becarios","funcion_id_4":"estudiantes","funcion_id_5":"personales_tecnico_de_apoyo","funcion_id_6":"co_directores","funcion_id_-1":"unkowns"}
    df_counts.rename(columns = function_mapping,inplace = True)
    df_counts["total_personel"] = df_counts.drop(columns = ["proyecto_id"]).sum(axis = 1)
    data = pd.merge(data, df_counts, on='proyecto_id', how='inner') 
    return data


In [5]:
def get_refs(data,ref_folder,refs):    
    current_data = pd.read_csv(f"{ref_folder}ref_{refs[0]}.csv",delimiter = ";")
    id_mapping = current_data[["disciplina_id","gran_area_descripcion"]].set_index("disciplina_id").to_dict()["gran_area_descripcion"]
    data["disciplina_id"] = data['disciplina_id'].replace(id_mapping)
    
    
    current_data = pd.read_csv(f"{ref_folder}ref_{refs[1]}.csv",delimiter = ";")
    id_mapping = current_data.set_index("id").to_dict()["descripcion"]
    data["estado_id"] = data['estado_id'].replace(id_mapping)
    
    current_data = pd.read_csv(f"{ref_folder}ref_{refs[2]}.csv",delimiter = ";")
    id_mapping = current_data[["moneda_id","moneda_desc"]].set_index("moneda_id").to_dict()["moneda_desc"]
    data["moneda_id"] = data['moneda_id'].replace(id_mapping)

    return data

In [6]:
data = get_main_data(main_folder)

In [7]:
data = get_extra(data,extra_folder,extras)

In [8]:
data = get_refs(data,ref_folder,refs)

In [9]:
data.to_csv("data.csv")